In [2]:
import pandas as pd
from prophet import Prophet

# Load the dataset
df = pd.read_csv('powerconsumption.csv', sep=',', parse_dates=['Datetime'])

# Columns representing power consumption in different zones
power_columns = ['PowerConsumption_Zone1', 'PowerConsumption_Zone2', 'PowerConsumption_Zone3']

# Forecasting function using Prophet
def prophet_forecast(data, column, future_steps):
    # Prepare data for Prophet: Rename columns to 'ds' and 'y' as required by Prophet
    prophet_data = data[['Datetime', column]].rename(columns={'Datetime': 'ds', column: 'y'})
    
    # Create and fit the model
    model = Prophet(uncertainty_samples=0)  # Disable uncertainty intervals if not needed
    model.fit(prophet_data)
    
    # Create a dataframe for future predictions (10-minute intervals)
    future = model.make_future_dataframe(periods=future_steps, freq='10min')
    
    # Make predictions
    forecast = model.predict(future)
    
    return forecast[['ds', 'yhat']]

# Forecast for the next 24 hours with 10-minute intervals (24 * 6 = 144 steps)
future_steps = 24 * 6

# Dictionary to store the forecasts for each zone
forecasts = {}
for column in power_columns:
    # Check for missing data and handle it if necessary
    if df[column].isna().sum() > 0:
        print(f"Warning: {column} contains missing values, handling them.")
        df[column].fillna(method='ffill', inplace=True)  # Forward fill as an example
    
    # Generate the forecast for each zone
    forecasts[column] = prophet_forecast(df, column, future_steps)

# Create a dataframe to hold the forecast results for all zones
future_df = pd.DataFrame()
future_df['Datetime'] = forecasts[power_columns[0]]['ds']  # Use the datetime from one of the forecasts
for column in power_columns:
    future_df[column] = forecasts[column]['yhat']

# Set the 'Datetime' column as the index
future_df.set_index('Datetime', inplace=True)

# Extract the forecast for the next 24 hours
future_df = future_df.iloc[-future_steps:]

# Display the forecast
print("Forecast for the next 24 hours")
print(future_df)


21:12:34 - cmdstanpy - INFO - Chain [1] start processing
21:12:58 - cmdstanpy - INFO - Chain [1] done processing
21:13:00 - cmdstanpy - INFO - Chain [1] start processing
21:13:35 - cmdstanpy - INFO - Chain [1] done processing
21:13:36 - cmdstanpy - INFO - Chain [1] start processing
21:14:08 - cmdstanpy - INFO - Chain [1] done processing


Forecast for the next 24 hours
                     PowerConsumption_Zone1  PowerConsumption_Zone2  \
Datetime                                                              
2017-12-31 00:00:00            26356.564146            22243.997519   
2017-12-31 00:10:00            25695.372651            21806.979403   
2017-12-31 00:20:00            25080.145416            21395.857010   
2017-12-31 00:30:00            24512.915434            21012.044204   
2017-12-31 00:40:00            23994.608530            20656.315939   
...                                     ...                     ...   
2017-12-31 23:10:00            30027.616603            25097.014045   
2017-12-31 23:20:00            29231.644205            24602.733100   
2017-12-31 23:30:00            28455.531336            24118.081237   
2017-12-31 23:40:00            27706.452913            23647.403493   
2017-12-31 23:50:00            26990.693830            23194.543241   

                     PowerConsumption_Zone3  

In [ ]:
z